# <center> 1. Structured code </center>

We are going now to put together all different pieces to have a flexible program. The method combining the different pieces will be defined by the following function
```
function mcising(I::NTuple{N,Int},β::T;  # linear size of the lattice + temperature
                 h::Vector{T}=T[], # external field
                 x0::Vector{Int}=rand([-1,1],prod(I)), # initial configuration 
                 nterm::Int=100,  # number of mcsweeps for thermalization
                 nmeas::Int=1000, # number of measurments
                 nsweep::Int=100, # number of mcsweeps between measurements
                 verbose::Bool=false) where {T<:Real,N}
```

The body of the program should be something like

```
for t in 1:nterm   # thermalization
    onemcsweep!(ising)
end

for n in 1:nmeas # loop for measurements   
    for t in 1:nsweep # number of mcsweeps between measurements
      onemcsweep!(ising)
    end
    measures!(output,ising)
end
```
where we we define a `struct` in which we will store our measurements:

```
mutable struct Measures{T<:AbstractFloat}
    it::Int # it contains the iteration step 
    β::T    # the inverse temperature
    ene::Vector{T}  # a vector of length nmeas containing the energy measured at time it 
    mag::Vector{T}  # a vector of length nmeas containing the magnetization measured at time it 
    conf::Vector{BitArray{1}} # a vector of length nmeas containing the spin configuration measured at time it 
end
```

We encode the spin configuration into a `BitArray{1}` using the following constructor. If `x` is an instance of type `Ising`, we can compute a Bool representation of the configuration as

```
BitArray(map(spin2bool,x.spin))
```
where 
```
spin2bool = x -> x > 0
```

# <center> 2. Establishing the thermalization time </center>

A practical concern is how to establish the thermalization time. To do so we perform a logarithmic block division of data.
Suppose we have `nmeas` data. We divide the data into blocks in the following way:
* block 1 - data from nmeas/2 + 1 to nmeas
* block 2 - data from nmeas/4 + 1 to nmeas/2
* block 3 - data from nmeas/8 + 1 to nmeas/4
* .... until there is at least 1 datum

Write a function 

```
function blockmean(data::AbstractArray)
```
which perform the mean over the blocks and returns three vectors: $t, \mu, \sigma$ of length nblocks containing:
1. the time at which the measurement has been performed (taken as the midpoint of the block)
2. the value of the mean of the data on the block
3. the std of the mean on computed on the block

# <center> 3. Establishing the equilibrium autocorrelation time </center>

To understand how frequently we should measure our observable, i.e. how to fix the `nsweep` number of mcsweeps between measurements, we should have an idea about how fast the system forgets its previous configuration at equilibrium. To do so we can measure the equilibrium autocorrelation function of any observable of the spin configuration $O(t):=O(s_1(t),\dots,s_N(t))$ in the following way:
$$
C_O(\Delta t) = \frac{1}{T_\mathrm{meas} -\Delta t}\sum_{t=1}^{T_\mathrm{meas} -\Delta t} O(t+\Delta t)O(t) - 
\left( \frac{1}{T_\mathrm{meas}-\Delta t}\sum_{t=1}^{T_\mathrm{meas}-\Delta t } O(t+\Delta t) \right)
\left( \frac{1}{T_\mathrm{meas}-\Delta t}\sum_{t=1}^{T_\mathrm{meas} -\Delta t} O( t)\right)
$$

Measure the autocorrelation time for the magnetization $m=\sum_{i=1}^N s_i$ and the energy over time at aquilibrium.